# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [6]:
#pip install hvplot

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,dt
0,0,zyryanka,65.7500,150.8500,-27.93,99,36,2.35,RU,1667766998
1,1,busselton,-33.6500,115.3333,13.60,84,79,4.51,AU,1667766643
2,2,rikitea,-23.1203,-134.9692,22.15,73,56,6.16,PF,1667766999
3,3,banda aceh,5.5577,95.3222,25.47,80,100,2.06,ID,1667766999
4,4,vaini,-21.2000,-175.2000,26.09,69,20,2.57,TO,1667767000


In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,dt
0,0,zyryanka,65.7500,150.8500,-27.93,99,36,2.35,RU,1667766998
1,1,busselton,-33.6500,115.3333,13.60,84,79,4.51,AU,1667766643
2,2,rikitea,-23.1203,-134.9692,22.15,73,56,6.16,PF,1667766999
3,3,banda aceh,5.5577,95.3222,25.47,80,100,2.06,ID,1667766999
4,4,vaini,-21.2000,-175.2000,26.09,69,20,2.57,TO,1667767000


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
#configure gmaps
gmaps.configure(api_key=g_key)

#get locations and humidity
city_locations = city_data_df[['Lat', 'Lng']].astype(float)
humidity = city_data_df['Humidity'].astype(float)

#format layout of map
layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'}

#add heatmap
fig = gmaps.figure(map_type='HYBRID', layout=layout)

heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

In [13]:
%%capture --no-display

# Configure the map plot
#gmaps.configure(api_key=g_key)
gmaps.configure(api_key=g_key)

locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure/map
fig


Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"]>20) & (city_data_df["Max Temp"]<29) & (city_data_df["Humidity"]<40) & (city_data_df["Wind Speed"]<5) &
                                    (city_data_df["Cloudiness"]<5)]

# Display sample data
print(ideal_weather_df.head())
ideal_weather_df.count()

     City_ID          City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
101      101     taoudenni  22.6783   -3.9836     26.74        12           0   
111      111     kongoussi  13.3258   -1.5347     27.32        16           1   
112      112        blythe  33.6103 -114.5964     26.69        20           0   
170      170  port augusta -32.5000  137.7667     20.60        38           0   
239      239      araouane  18.9048   -3.5265     26.47        16           0   

     Wind Speed Country          dt  
101        4.99      ML  1667766735  
111        3.59      BF  1667767023  
112        1.54      US  1667766737  
170        1.41      AU  1667767039  
239        3.95      ML  1667766714  


City_ID       8
City          8
Lat           8
Lng           8
Max Temp      8
Humidity      8
Cloudiness    8
Wind Speed    8
Country       8
dt            8
dtype: int64

### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City","Country","Max Temp","Lat","Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)


,City,Country,Max Temp,Lat,Lng,Hotel Name
101,taoudenni,ML,26.74,22.6783,-3.9836,
111,kongoussi,BF,27.32,13.3258,-1.5347,
112,blythe,US,26.69,33.6103,-114.5964,
170,port augusta,AU,20.60,-32.5000,137.7667,
239,araouane,ML,26.47,18.9048,-3.5265,
303,aswan,EG,22.61,24.0934,32.9070,
400,pecos,US,28.50,31.4229,-103.4932,
508,dosso,NE,25.50,13.0490,3.1937,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
#radius = 10000
params = {
     "radius": 50000,
    "rankby": "prominence",
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key,}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #params["filter"] = # YOUR CODE HERE
    params["location"] = f"{lat},{lng}"

    #params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    #base_url = "https://api.geoapify.com/v2/places"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # Make and API request using the params dictionaty
    #name_address = # YOUR CODE HERE

    # Convert the API response to JSON format
    #name_address = # YOUR CODE HERE
    #print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    print (response)
    #break
    #results = response['results']
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
    # Print end of search once searching is completed
print("-------End of Search-------")

# Display sample data
hotel_df


Starting hotel search
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
taoudenni - nearest hotel: No hotel found
{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 13.0847909, 'lng': -1.0800828}, 'viewport': {'northeast': {'lat': 13.08610702989272, 'lng': -1.078732620107278}, 'southwest': {'lat': 13.08340737010728, 'lng': -1.081432279892722}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Hôtel Kaziendé', 'opening_hours': {'open_now': True}, 'photos': [{'height': 432, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/100850914345116725013">A Google User</a>'], 'photo_reference': 'AW30NDyz8UtXvDeSMR8qIiJrY4xyMkFpX0Omg5QWvaUpSnpcxdMvC31gkPoz3VxK_4hhF7FfmUbopBbSX52E2lS-rNcoq1VrXfOMFIl-9YI81nn_bHa0nbdjA6H3hbLj4EeNYEDGDVGz7B_j

{'html_attributions': [], 'next_page_token': 'AW30NDyqZRmGalhggLS-Zuv7bnhKRAF40GPvfU48ZaZH0frN6i_h13uh_RrSMbWxzdwYNhzZnXYJmUc9n4FiNf3zS00oOZi0WB8KF6yFR5sn2zASQTo5jF20CRBVGWFhH6TxHsg7yFFtIA18qSAFWN_nVbY5gOLNF99rGH2X_oRsiSNYUi-ZxBOZ81DIMwYY_XSSDLD8OlQ250p6aDhJbttbHwVt8V4kiWZ8Urtf31vvgYMA_tiR_lzIk0qxccgxuRO4pGoBjiFyMow_ycdmpGgP0s3Aw5hwII1JV0myMxmfLhsqt5ILKS1uQrBpllTKW4PO6EzRN2FMK_cYGaxBU7T-peBRWZu3D3Fzv6UY2zd4UlreeZtJhfhEw-UNGTk2oNGACoNaDfqiWJbPftNlgJcXwZZ2--terOdopvJUgPQDyvhZjb3SYqVQP14SfZxtj7DFjwnq', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -32.82690840000001, 'lng': 138.1867009}, 'viewport': {'northeast': {'lat': -32.82557217010728, 'lng': 138.1880442798927}, 'southwest': {'lat': -32.82827182989272, 'lng': 138.1853446201073}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'N

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
araouane - nearest hotel: No hotel found
{'html_attributions': [], 'next_page_token': 'AW30NDzrxYKTwfDiW8qzhvVGIiQEMFWWDuPMuZWVBUrRUp4gWz1-to_NflZI25tuetc5w0L86eEG_JzD2zumrddOByVedJsWq4kuyVwti2zxIiW-5__cjiZq7eO-Ot0DMM7RTIsIcjcuEuSLQpqhKT2BXekYhV8A9Sy0docn7JG-4jUK4cDj26rRVirrTtyHg1FG2od2aQSValX07Rgp8f2Jue38Qcgg705hm_QsHmDO0Hwz7DUhQbkayHFIXcLCezQk8-P05lIeGFySTUoAW1D4dILPM2dC1VUKZpTqsWbXlqpfznfrM3gJUMGQpAvTpTCl8Z120qALdoDT1bXVde6JoZQ3q0ulSu8bU52vbgMzOl8w5K1Vuqi5shqozT6LnA_Hsbud94BQCSAxrlvICPIBvPR3jKckinJrQqblcd2VJ7oVO26JEtVEgSKOxJxCVdubLoCE', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 24.0824179, 'lng': 32.8881741}, 'viewport': {'northeast': {'lat': 24.08376617989273, 'lng': 32.88952377989273}, 'southwest': {'lat': 24.08106652010728, 'lng': 32.88682412010728}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'ico

{'html_attributions': [], 'next_page_token': 'AW30NDxZzhXaA_66gFW309PeyHp5ZmscJkxiWffHcKvTppoaS0AK_v4h3RaT6Vvr2JBuxQQGreaqR_0fTONo0Ftbx3N9-PMWOqjjifYAAaZzNvPDF1k-_XFc2vWFe_pcM6TyagaXvJv6k9Oluyjb7bHavDUjA1O8wqoSeitRdJL6aM64mQt3ZaQ3DJbDZwXBHmdG7642YATRYfdBxxm0K2sdfwJ2ylLOCPHzO0opiDSUS9rF4N7ox6-HZ8ObDjFWnsgZaP1x0ip_XjGk9L3jUKl4mw5RJwS5QPobw6Gr2MxpK7OUMr9wLGQjinkn1oKYHLLtRVL0A_cgTdyjfhPRDt63KOP4MqFbX65CYoQ6OXesQXYOgLYBTVid84bh7alS-KCkZB7c97NOhD9QRkZ4eM_FUs5QfdW8gzNhpg6D1wrC0tHo9uapM6PXaIqzyfqnJl_vYDUo', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 31.8326003, 'lng': -103.9148866}, 'viewport': {'northeast': {'lat': 31.83378747989272, 'lng': -103.9135780201073}, 'southwest': {'lat': 31.83108782010728, 'lng': -103.9162776798927}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'The Arma

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 13.010498, 'lng': 3.1980862}, 'viewport': {'northeast': {'lat': 13.01179792989272, 'lng': 3.198867129892723}, 'southwest': {'lat': 13.00909827010728, 'lng': 3.196167470107278}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Hotel Toubal', 'photos': [{'height': 720, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104213541844252076877">A Google User</a>'], 'photo_reference': 'AW30NDzf85Eb5KKzybdxp0GBHpA3y_-TnAGaH1umDGJXDxqIrsScO7paYhbEa6Xoolve3ctKBullT0ALjCxLIw9YubwSBdzngUPgaXRaEf8PuhTjpFLjjXtYxo4E5x3kfH-DkQESp1FTqvTtYab8RT0-fevsj1R5wN1ZaI46e9hM3U9w-DCu', 'width': 960}], 'place_id': 'ChIJeWCb_SEPzxER8tMptFnaDTk', 'plus_code': {'compound_code': '256X+56 Dosso, Niger', 'global_code': '7F552

,City,Country,Max Temp,Lat,Lng,Hotel Name
101,taoudenni,ML,26.74,22.6783,-3.9836,No hotel found
111,kongoussi,BF,27.32,13.3258,-1.5347,Hôtel Kaziendé
112,blythe,US,26.69,33.6103,-114.5964,"Holiday Inn Express & Suites Blythe, an IHG Hotel"
170,port augusta,AU,20.60,-32.5000,137.7667,North Star Hotel
239,araouane,ML,26.47,18.9048,-3.5265,No hotel found
303,aswan,EG,22.61,24.0934,32.9070,Sofitel Legend Old Cataract Aswan
400,pecos,US,28.50,31.4229,-103.4932,The Armadillo Hotel Orla
508,dosso,NE,25.50,13.0490,3.1937,Hotel Toubal


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
hotel_df = hotel_df[hotel_df['Hotel Name']!= "No hotel found"]
                
hotel_df = hotel_df.rename(columns={"Latitude": "Lat", "Longitude": "Lng"})
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
111,kongoussi,BF,27.32,13.3258,-1.5347,Hôtel Kaziendé
112,blythe,US,26.69,33.6103,-114.5964,"Holiday Inn Express & Suites Blythe, an IHG Hotel"
170,port augusta,AU,20.60,-32.5000,137.7667,North Star Hotel
303,aswan,EG,22.61,24.0934,32.9070,Sofitel Legend Old Cataract Aswan
400,pecos,US,28.50,31.4229,-103.4932,The Armadillo Hotel Orla
508,dosso,NE,25.50,13.0490,3.1937,Hotel Toubal


In [26]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""


In [27]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
%%capture --no-display

# Configure the map plot
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=max_temp,
                                 dissipating=False,
                                 max_intensity=300,
                                 point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display the map
fig


Figure(layout=FigureLayout(height='420px'))